# The Model

## setup

In [1]:
## imports
#  modules
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import torch
import os

# custom code
import dask_addons
from open_data import open_data_dir, data_dir_to_zarr
from neural_net import DNN, geoDataset
from open_fire_data import FlattenedTruthTable

warnings.filterwarnings('ignore')

2025-08-05 15:51:01,425 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


In [ ]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55256,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:55279,Total threads: 2
Dashboard: http://127.0.0.1:55281/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:55259,


2025-08-05 16:34:00,726 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2166s; attempting restart: <WorkerState 'tcp://127.0.0.1:55276', name: 2, status: running, memory: 0, processing: 0>
2025-08-05 16:34:00,730 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2166s; attempting restart: <WorkerState 'tcp://127.0.0.1:55279', name: 0, status: running, memory: 0, processing: 0>
2025-08-05 16:34:00,731 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2166s; attempting restart: <WorkerState 'tcp://127.0.0.1:55280', name: 3, status: running, memory: 0, processing: 0>
2025-08-05 16:34:00,732 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2166s; attempting restart: <WorkerState 'tcp://127.0.0.1:55285', name: 1, status: running, memory: 0, processing: 0>
2025-08-05 16:34:01,686 - distributed.nanny - WARNING - Restarting worker
2025-08-05 16:34:01,691 - distributed.nanny - WARNING - Restarting worker
2025-08-05 16:34:01,

In [3]:
DATA_DIR_PATH = '../data/_ZARR_FILES'

data_path = os.path.join(DATA_DIR_PATH, 'alaska_full.zarr')
prior_data_path = os.path.join(DATA_DIR_PATH, 'alaska_prior.zarr')

data = xr.open_zarr(data_path, decode_timedelta=False)
prior_data = xr.open_zarr(prior_data_path, decode_timedelta=False)

In [4]:
ds = dask_addons.FlattenedDaskDataset(data, prior_data)
ds.compute_resultant_speed()
ds.data

<xarray.Dataset> Size: 8GB
Dimensions:     (longitude: 131, time: 1886, step: 24, latitude: 38)
Coordinates:
  * longitude   (longitude) float64 1kB -153.7 -153.6 -153.5 ... -140.8 -140.7
  * latitude    (latitude) float64 304B 65.0 64.9 64.8 64.7 ... 61.5 61.4 61.3
    number      int64 8B ...
    surface     float64 8B ...
  * step        (step) float64 192B 1.0 2.0 3.0 4.0 5.0 ... 21.0 22.0 23.0 24.0
    valid_time  (time, step) datetime64[ns] 362kB dask.array<chunksize=(943, 24), meta=np.ndarray>
  * time        (time) datetime64[ns] 15kB 2009-12-31 2010-01-01 ... 2014-12-31
Data variables:
    lai_lv      (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    lai_hv      (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    d2m         (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    t2m         (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    sp          (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    v10         (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    u10         (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    tp          (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
    ws10        (time, step, latitude, longitude) float32 901MB dask.array<chunksize=(236, 6, 10, 33), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-05T13:52 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
fire_data = FlattenedTruthTable(pd.read_csv('../data/_FIRE/alaska_range_csv/data.csv'))
fire_data.data

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,64.7,-147.0,308.9,1.0,1.0,2010-04-17,22.0,Terra,MODIS,68,6.03,281.6,12.4,D,0
1,64.8,-147.2,306.1,1.0,1.0,2010-04-18,23.0,Aqua,MODIS,56,6.03,284.9,8.2,D,0
2,63.9,-145.6,315.5,2.6,1.5,2010-04-23,21.0,Aqua,MODIS,76,6.03,279.9,65.4,D,0
3,63.9,-145.6,318.5,2.6,1.5,2010-04-23,21.0,Aqua,MODIS,78,6.03,281.9,76.0,D,0
4,63.9,-145.6,302.1,2.0,1.4,2010-04-23,23.0,Terra,MODIS,50,6.03,282.6,18.0,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6782,61.8,-142.6,301.3,1.0,1.0,2014-07-21,21.0,Terra,MODIS,0,6.03,289.8,8.4,D,0
6783,61.4,-150.1,301.1,1.0,1.0,2014-08-31,23.0,Aqua,MODIS,39,6.03,289.6,4.6,D,0
6784,61.4,-150.1,300.1,1.1,1.1,2014-09-02,8.0,Terra,MODIS,17,6.03,280.4,7.8,N,0
6785,64.9,-147.8,307.6,2.0,1.4,2014-09-28,7.0,Terra,MODIS,72,6.03,271.7,37.7,N,0


In [ ]:
for i in range(1000000000000):
    if fire_data[i] != 0:
        print(f'{i = }, {fire_data[i] = }')

    if i%10000 == 0:
        print(f'{i = }')

code in md so i don't accidentally run it; took ~15min to convert full dataset to zarr (no editing done)
```
ddpath = os.path.join('../data', 'alaska_main')
target_path = os.path.join(DATA_DIR_PATH, 'alaska_full.zarr')

data_dir_to_zarr(ddpath, target_path)
```

## training


In [ ]:
from random import random
model = DNN(input_size = 7)

model.train()

X = torch.tensor(np.array([[random() for _ in range(7)] for i in range(20)]), dtype=torch.float32)

model(X)